# Grand canonical ensemble Monte Carlo

In this example, a short grand canonical Monte Carlo simulation of Lennard Jones particles is conducted.

In [1]:
import sys
import unittest
import feasst
sys.path.insert(0, feasst.install_dir() + '/plugin/system/tutorial/')
import lj_system
sys.path.insert(0, feasst.install_dir() + '/plugin/monte_carlo/tutorial/')
import analyze

class TestMonteCarlo2LJGCMC(unittest.TestCase):
    """Test a grand canonical ensemble Lennard Jones Monte Carlo simulation"""
    def test(self):
        """Compute the average number of particles and assert that it is greater than 0"""
        monte_carlo = feasst.MonteCarlo()
        monte_carlo.set(lj_system.system(lj_system.configuration(box_length=8)))
        monte_carlo.set(feasst.MakeMetropolis(feasst.args(
            {"beta": str(1./1.5), "chemical_potential": "-8.352321"})))
        monte_carlo.add(feasst.MakeTrialTranslate(feasst.args(
            {"weight": "1.", "tunable_param": "2."})))
        # add an insertion/deletion trial attempt
        feasst.add_trial_transfer(monte_carlo, feasst.args({"weight": "1.", "particle_type": "0"}))
        steps_per = int(1e3)
        analyze.add(monte_carlo, steps_per)

        # Add an Analyze which computes the average number of particles.
        # Just before adding, store the number of existing Analyzers in order to remember the
        # index of the newly added Analyze.
        analyze_index = monte_carlo.num_analyzers()
        monte_carlo.add(feasst.MakeNumParticles(feasst.args(
            {"steps_per_write": str(steps_per), "file_name": "gcmc_num_particles.txt"})))

        # peform a short simulation
        monte_carlo.attempt(int(1e5))

        # assert that particles were added during the simulation
        self.assertTrue(monte_carlo.analyze(analyze_index).accumulator().average() > 0)


In [2]:
%%time
unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestMonteCarlo2LJGCMC)
Compute the average number of particles and assert that it is greater than 0 ... 

CPU times: user 171 ms, sys: 4.08 ms, total: 175 ms
Wall time: 175 ms


ok

----------------------------------------------------------------------
Ran 1 test in 0.172s

OK
